In [1]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [2]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

def compute_spread(p1, p2, tf, x):
    data1 = get_rates(p1, tf, x)
    data2 = get_rates(p2, tf, x)
    merged = data1.join(data2, lsuffix="_x", rsuffix="_y")
    spread = merged['close_x'] - merged['close_y']
    return spread.dropna()

def adf_test(spread):
    '''Runs ADF test on a spread series'''
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

# Correlation Mean Reversion

If correlation diverges between two pairs of assets, price of the two assets will move oppositely. If correlation is mean reverting, you can profit from the correlation reverting back to its mean. Identify the leading and lagging pair for both high and low correlation. 
- Go long on the lagging
- Go short on the leader

In [55]:
def get_pair_correlations(symbol1, symbol2, window):
    s1 = str(symbol1)
    s2 = str(symbol2)
    symbol1 = get_rates(symbol1, mt5.TIMEFRAME_H4, 5000)
    symbol2 = get_rates(symbol2, mt5.TIMEFRAME_H4, 5000)

    combined_df = pd.concat([symbol1['close'].rename(f'{s1}_close'),
                             symbol2['close'].rename(f'{s2}_close')], axis=1)

    window_size = window  # Change this to the size of the window you want
    combined_df['rolling_corr'] = combined_df[f'{s1}_close'].rolling(window=window_size).corr(combined_df[f'{s2}_close'])
    # combined_df['rolling_corr'].iloc[0:200].plot()
    combined_df[f'{s1}_return'] = combined_df[f'{s1}_close'].pct_change()
    combined_df[f'{s2}_return'] = combined_df[f'{s2}_close'].pct_change()
    combined_df['diff'] = combined_df[f'{s1}_return'] - combined_df[f'{s2}_return']
    combined_df['rolling_corr_returns'] = combined_df['rolling_corr'].rolling(window=window_size).corr(combined_df['diff'])
    combined_df['shifted_rolling_corr_returns'] = combined_df['rolling_corr_returns'].shift(1)
    return combined_df.dropna()

In [56]:
df = get_pair_correlations('EURUSD.a', 'GBPUSD.a', 5)
df

,EURUSD.a_close,GBPUSD.a_close,rolling_corr,EURUSD.a_return,GBPUSD.a_return,diff,rolling_corr_returns,shifted_rolling_corr_returns
time,,,,,,,,
2020-08-13 00:00:00,1.18067,1.30572,-0.114810,0.001807,0.001957,-0.000149,-0.068311,-0.420598
2020-08-13 04:00:00,1.18012,1.30502,0.972809,-0.000466,-0.000536,0.000070,-0.121124,-0.068311
2020-08-13 08:00:00,1.18384,1.30861,0.993513,0.003152,0.002751,0.000401,0.477185,-0.121124
2020-08-13 12:00:00,1.18490,1.30980,0.998387,0.000895,0.000909,-0.000014,0.303580,0.477185
2020-08-13 16:00:00,1.18174,1.30796,0.957012,-0.002667,-0.001405,-0.001262,-0.009083,0.303580
...,...,...,...,...,...,...,...,...
2023-10-25 00:00:00,1.05926,1.21662,0.999116,0.000170,0.000452,-0.000282,0.822415,0.834646
2023-10-25 04:00:00,1.06062,1.21750,0.952787,0.001284,0.000723,0.000561,-0.941762,0.822415
2023-10-25 08:00:00,1.05851,1.21384,0.825949,-0.001989,-0.003006,0.001017,-0.746943,-0.941762


In [63]:
grangercausalitytests(df[['shifted_rolling_corr_returns', 'EURUSD.a_return']], 10)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=22.6179 , p=0.0000  , df_denom=4987, df_num=1
ssr based chi2 test:   chi2=22.6315 , p=0.0000  , df=1
likelihood ratio test: chi2=22.5804 , p=0.0000  , df=1
parameter F test:         F=22.6179 , p=0.0000  , df_denom=4987, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=11.4486 , p=0.0000  , df_denom=4984, df_num=2
ssr based chi2 test:   chi2=22.9202 , p=0.0000  , df=2
likelihood ratio test: chi2=22.8677 , p=0.0000  , df=2
parameter F test:         F=11.4486 , p=0.0000  , df_denom=4984, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=7.5295  , p=0.0001  , df_denom=4981, df_num=3
ssr based chi2 test:   chi2=22.6204 , p=0.0000  , df=3
likelihood ratio test: chi2=22.5692 , p=0.0000  , df=3
parameter F test:         F=7.5295  , p=0.0001  , df_denom=4981, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=5.7377  , p=0.

{1: ({'ssr_ftest': (22.617927995890252, 2.0320176062912534e-06, 4987.0, 1),
   'ssr_chi2test': (22.631534128632918, 1.9623718419532105e-06, 1),
   'lrtest': (22.58036750069914, 2.0153355747001994e-06, 1),
   'params_ftest': (22.61792799588909, 2.032017606292979e-06, 4987.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (11.448622297486368, 1.0947447636104042e-05, 4984.0, 2),
   'ssr_chi2test': (22.920215345970902, 1.05423753708887e-05, 2),
   'lrtest': (22.8677265894421, 1.082271605074158e-05, 2),
   'params_ftest': (11.448622297486175, 1.0947447636110595e-05, 4984.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (7.529545399299423, 5.035895794104281e-05, 4981.0, 3),
   'ssr_chi2test': (22.620380918513664, 4.8450649377310915e-05, 3),
   'lrtest': (22.569244199894456, 4.965362505352943e-05, 3),
   'params_ftest': (7.529545399298981, 5.035895794105819e-05, 4981.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0.,